In [1]:
# Import libraries
import pandas as pd
import numpy as np
import ipywidgets as wdg
from plotly_default import go, graph_config, sel_trace
from plotly.subplots import make_subplots

In [2]:
log_df = pd.read_csv('./Data/Log_data_combined.csv')
species_types=pd.read_csv('./Controls/Species_types.csv')

In [3]:
cat_symbols = {'Mudstone': 'circle',
               'Carbonate': 'star',
               'Pervasively pyritised regions': 'square',
               'Bed 22': 'diamond'}
categories = list(cat_symbols.keys())

def init_biv_plot():
    plot = go.FigureWidget()
    
    for category in categories:   
        plot.add_trace(go.Scatter(mode='markers',
                                  marker_line={'color': 'Black', 'width': 1},
                                  marker_symbol=cat_symbols[category],
                                  marker_coloraxis='coloraxis',
                                  name=category))
        
    # Add diagonal line
    plot.add_shape(type='line', x0=-100, y0=-100, x1=100, y1=100, visible=False, line_width=0.5, name='Diagonal')

    plot.update_layout(height=650, width=800,
                       showlegend = False,
                       legend=dict(orientation="h",
                                   yanchor="bottom",
                                   y=1.01,
                                   xanchor="left",
                                   x=0,
                                   itemsizing='constant'),
                       coloraxis=dict(colorscale='viridis_r', colorbar_ticks='outside'))
    
    return plot

In [4]:
fig1 = init_biv_plot()

@wdg.interact(C_to_py = wdg.FloatSlider(value=4, min=0, max=8, step=0.001),
              C_to_N = wdg.FloatSlider(value=106/16, min=0, max=40),
              progress = wdg.FloatSlider(value=1, min=0, max=1, step = 0.001))
def update_C_plot(C_to_py, C_to_N, progress):
    
    with fig1.batch_update():
        for category in categories:
            cat_df = log_df[log_df.category == category]
            
            C_mes = np.array(cat_df['TOC'])
            N_mes = np.array(cat_df['TN'])
            py_mes = np.array(cat_df['Fe_py'])
            d34S_mes = np.array(cat_df['d34S'])
            
            C_N_mass = C_to_N * 12.011/14.007
            
            dC_i = py_mes * C_to_py * 12.011/55.845
            C_i = C_mes + dC_i
            N_i = N_mes + dC_i/C_N_mass
            
            dC = dC_i * progress
            Cs = C_i - dC
            Ns = N_i - dC/C_N_mass
            
            pys = py_mes * progress
            
            Cs_to_Ns = Cs / Ns

            graph_texts = []
            i = 0
            for d in cat_df.iterrows():
                graph_texts.append(d[1]['name'] + '<br>' +
                                   '[Fe<sub>py</sub>] (wt%)' + ': {:.2f}<br>'.format(pys[i]) +
                                   'δ<sup>34</sup>S<sub>py</sub> (‰)' + ': {:.2f}'.format(d34S_mes[i]))
                i += 1

            cat_trace = sel_trace(fig1, category)
            cat_trace.x=Cs
            cat_trace.y=Cs_to_Ns
            cat_trace.marker.size=pys
            cat_trace.marker.sizemin=4
            cat_trace.marker.sizeref=3*np.mean(py_mes/(6.**2))
            cat_trace.marker.color=d34S_mes
            cat_trace.text=graph_texts
            
        # Only concentration axes should go to zero
        fig1.layout.xaxis.rangemode = 'tozero'

        fig1.layout.xaxis.title = 'TOC (wt%)'
        fig1.layout.yaxis.title = 'TOC/TN'
        fig1.layout.coloraxis.colorbar.title = 'δ<sup>34</sup>S<sub>py</sub> (‰)'
        
fig1

interactive(children=(FloatSlider(value=4.0, description='C_to_py', max=8.0, step=0.001), FloatSlider(value=6.…

FigureWidget({
    'data': [{'marker': {'color': array([-21.9907799 , -16.03659904, -13.28036687, -15.8747187 …

In [10]:
fig2 = init_biv_plot()

for category in categories:
    cat_df = log_df[log_df.category == category]
    
    cat_trace = sel_trace(fig2, category)
    cat_trace.x = cat_df['TOC']
    cat_trace.y = (cat_df['TOC']/12.011)/(cat_df['TN']/14.007)
    
fig2.add_trace(go.Scatter(mode='lines', line_color='black', name='model'))

fig2.layout.xaxis.rangemode = 'tozero'

fig2.layout.xaxis.title = 'TOC (wt%)'
fig2.layout.yaxis.title = 'TOC/TN'
    

@wdg.interact(TN_silicate = wdg.FloatSlider(value=0.05, min=0, max=1, step=0.001, layout=wdg.Layout(width='60%')),
              CN_kerogen = wdg.FloatSlider(value=50, min=0, max=100, step = 0.001, layout=wdg.Layout(width='60%')))
def Kipp_model(TN_silicate, CN_kerogen):
    
    with fig2.batch_update():
        
        TOCs = np.arange(1,20, 0.05)
        
        CNs = (TOCs/12.011)/((TN_silicate+((TOCs*14.01)/(CN_kerogen*12.01)))/14.007)
        
        tr = sel_trace(fig2, 'model')

        tr.x = TOCs
        tr.y=CNs
        
fig2

interactive(children=(FloatSlider(value=0.05, description='TN_silicate', layout=Layout(width='60%'), max=1.0, …

FigureWidget({
    'data': [{'marker': {'coloraxis': 'coloraxis', 'line': {'color': 'Black', 'width': 1}, 'sym…